In [10]:
import time
import tensorflow as tf
import numpy as np
import pygame
import random
import itertools
from collections import defaultdict
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, RMSprop
from IPython.display import clear_output

#dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
print(tf.__version__)

2.2.0


In [21]:
class snek:

    def __init__(self, size):
        self.size = size
        self.rewards = None
        self.action_space_size = 4
        self.actions = [0, 1, 2, 3]
        self.cstate = None
        self.dis = None
        self.snake = None
        self.head = None
        self.food = None
        self.scolor = (0, 0, 255)
        self.fcolor = (255, 0, 0)
        self.len = 1
    
    def reset(self):
        self.snake = []
        state = np.zeros((self.size, self.size))
        size = self.size
        x = random.randint(0, size - 1)
        y = random.randint(0, size - 1)
        foodx = random.randint(0, size - 1)
        foody = random.randint(0, size - 1) 
        self.food = (foodx, foody)
        self.head = (x, y)
        self.snake.append((x, y))
        state[x][y] = 1
        state[foodx][foody] = -1
        self.cstate = state
        return state
    
    def step(self, action):
        # Left = 0 Right = 1 Up = 2 Down = 3
        done = False
        reward = 0
        dx, dy = 0, 0
        if action == 0:
            dx, dy = -1, 0
        elif action == 1:
            dx, dy = 1, 0
        elif action == 2:
            dx, dy = 0, -1
        elif action == 3:
            dx, dy = 0, 1
        
        x, y = self.head
        if x>=self.size or x<0 or y>=self.size or y<0:
            done = True
        x += dy
        y += dx
        self.head = (x,y)
        self.snake.append(self.head)

        if self.head == self.food:
            reward = 10
            self.len += 1
            self.food = (random.randint(0, self.size - 1), random.randint(0, self.size - 1))

        elif self.len < len(self.snake):
            del self.snake[0]
        
        for x in self.snake[:-1]:
            if x == self.head:
                done = True
        
        state = np.zeros((self.size, self.size))
        for x in self.snake:
            state[x[0]][x[1]] = 1
        state[self.food[0]][self.food[1]] = -1
        self.cstate = state

        if done == True:
            reward = -100
        return state, reward, done
        
        

        

In [22]:
env = snek(64)

In [23]:
pygame.init()
dis = pygame.display.set_mode((640, 640))
pygame.display.set_caption("AI Learns to play Snake")
icon = pygame.image.load('/home/veeraraghavan/Desktop/DQL-Snake/Images/snake.png')
pygame.display.set_icon(icon)
gameover = False
state = env.reset()
while gameover != True:
    dis.fill((0, 0, 0))
    action = int(input())
    next_state, reward, done = env.step(action)
    if done:
        gameover = True
    print(reward)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            gameover = True
    for square in env.snake:
        pygame.draw.rect(dis, env.scolor, [square[1] * 10, square[0] * 10, 10, 10])
    pygame.draw.rect(dis, env.fcolor, [env.food[1] * 10, env.food[0] * 10, 10, 10])
    pygame.display.update()
    #time.sleep(1)
pygame.quit()


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-100


In [17]:
pygame.quit()
